# Defining suspected infection

The Sepsis 3 guidelines specify that among patients with suspected infection, a SOFA value >= 2 indicates sepsis. This notebook overviews the implementation of suspicion of infection in the MIMIC-III database.

The appendix of Seymour et al. details the algorithm used, which is:

* prescription of an antibiotic (excluding single dose for surgical patients)
* acquisition of a fluid culture

If these two occur in close temporal proximity, then this is seen as evidence that the clinician suspects infection in the patient.

First we import all the necessary libraries. This notebooks requires MIMIC-III v1.4 available in a PostgreSQL instance (you can specify the connection details below).

In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import sys

from IPython.display import display, HTML # used to print out pretty pandas dataframes

# colours for prettier plots
col = [[0.9047, 0.1918, 0.1988],
    [0.2941, 0.5447, 0.7494],
    [0.3718, 0.7176, 0.3612],
    [1.0000, 0.5482, 0.1000],
    [0.4550, 0.4946, 0.4722],
    [0.6859, 0.4035, 0.2412],
    [0.9718, 0.5553, 0.7741],
    [0.5313, 0.3359, 0.6523]];
marker = ['v','o','d','^','s','o','+']
ls = ['-','-','-','-','-','s','--','--']
%matplotlib inline

In [2]:
# create a database connection (you may need to update this)
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'public,mimiciii'
query_schema = 'set search_path to ' + schema_name + ';'
# Connect to local postgres version of mimic
con = psycopg2.connect(dbname=dbname, user=sqluser)
# you can also specify host and port in the above call

# Blood culture

Blood cultures are relatively simple to identify in MIMIC-III. The `microbiologyevents` contains all blood cultures made from patients along with sensitivities. 

The table is structured as follows: there is an `spec_type_desc` column which describes the specimen analyzed (usually blood, but can be any fluid e.g. urine, pleural, etc).

If a culture is negative (i.e. no bacteria grew), then the remaining columns are null.

If a culture is positive, then for each organism that grew, a row exists and the organism is named in `org_name`. Optionally, antibiotics can be tested for sensitivity, and these are listed in `ab_name`.

Here's an example.

In [3]:
query = query_schema + \
"""
SELECT hadm_id, TO_CHAR(charttime, 'HH:MM') as tm
, spec_type_desc, org_name, ab_name
, interpretation
FROM microbiologyevents
WHERE hadm_id = 145167
ORDER BY charttime;
"""

df = pd.read_sql_query(query, con)
df.head(n=6)

,hadm_id,tm,spec_type_desc,org_name,ab_name,interpretation
0,145167,09:08,SPUTUM,STAPH AUREUS COAG +,GENTAMICIN,S
1,145167,09:08,SPUTUM,STAPH AUREUS COAG +,OXACILLIN,S
2,145167,09:08,SPUTUM,STAPH AUREUS COAG +,ERYTHROMYCIN,R
3,145167,09:08,SPUTUM,STAPH AUREUS COAG +,PENICILLIN,R
4,145167,09:08,SPUTUM,STAPH AUREUS COAG +,LEVOFLOXACIN,S
5,145167,11:08,URINE,None,None,None


Note that we have excluded some columns from this display (in particular, dilution data for the antibiotic data is available).

Reading from left to right, at the top row, we can see:

* The patient's sputum was tested at 09:08
  * it was positive for staphylococcus aureus
  * multiple antibiotics were tested
  * bacteria was sensitive ('S') to gentamicin, oxacillin, and levofloxacin
  * bacteria was resistant ('R') to erythromycin and penicillin.
* The patient's urine was tested and was negative for bacterial growth

Using this logic, we can make a simple query that groups cultures together and assigns a flag indicating whether the culture was positive or negative (this is just for our information - the sepsis-3 criteria make no mention of positive vs. negative as a requirement).

In [4]:
query_me = """
select hadm_id
    , chartdate, charttime
    , spec_type_desc
    -- if organism is present, then this is a positive culture, otherwise it's negative
    , max(case when org_name is not null and org_name != '' then 1 else 0 end) as PositiveCulture
from microbiologyevents
group by hadm_id, chartdate, charttime, spec_type_desc
"""

We will later combine this with antibiotics usage to define suspected infection.

# Antibiotics usage

We can extract antibiotic usage from two sources:

* `prescriptions` table - prescribed medications
* `inputevents_mv` and `inputevents_cv` tables - intravenously delivered medications

Both of these approaches have upsides and downsides. 

For prescriptions, this is sourced from a hospital wide database, so it captures antibiotic prescriptions before ICU admission. However, as these are prescriptions and not administrations, we have no guarantee that the patient received the medication (though it is very likely in the case of antibiotics). We also only have a start *date* (i.e. the day of prescription), as the exact time of prescription is not available.

For the input events tables, these tables only contain antibiotics given intravenously. It's not unreasonable to assume that antibiotics given to patients admitted to the ICU will be almost entirely intravenously. We have much higher time resolution (to the hour) for antibiotic administration, but this data is only recorded in the ICU.

Since we have two possible sources of antibiotic usage, we will extract antibiotics using both and compare them at the end.

## Antibiotics - intravenous administration

Here we use the same logic, except we extract antibiotics from the `inputevents_mv` and `inputevents_cv` tables.

In [5]:
query_ab_ditems = \
"""
with di as
(
  select
    di.*
    , case
      when lower(label) like '%' || lower('adoxa') || '%' then 1
      when lower(label) like '%' || lower('ala-tet') || '%' then 1
      when lower(label) like '%' || lower('alodox') || '%' then 1
      when lower(label) like '%' || lower('amikacin') || '%' then 1
      when lower(label) like '%' || lower('amikin') || '%' then 1
      when lower(label) like '%' || lower('amoxicillin') || '%' then 1
      when lower(label) like '%' || lower('amoxicillin%clavulanate') || '%' then 1
      when lower(label) like '%' || lower('clavulanate') || '%' then 1
      when lower(label) like '%' || lower('ampicillin') || '%' then 1
      when lower(label) like '%' || lower('augmentin') || '%' then 1
      when lower(label) like '%' || lower('avelox') || '%' then 1
      when lower(label) like '%' || lower('avidoxy') || '%' then 1
      when lower(label) like '%' || lower('azactam') || '%' then 1
      when lower(label) like '%' || lower('azithromycin') || '%' then 1
      when lower(label) like '%' || lower('aztreonam') || '%' then 1
      when lower(label) like '%' || lower('axetil') || '%' then 1
      when lower(label) like '%' || lower('bactocill') || '%' then 1
      when lower(label) like '%' || lower('bactrim') || '%' then 1
      when lower(label) like '%' || lower('bethkis') || '%' then 1
      when lower(label) like '%' || lower('biaxin') || '%' then 1
      when lower(label) like '%' || lower('bicillin l-a') || '%' then 1
      when lower(label) like '%' || lower('cayston') || '%' then 1
      when lower(label) like '%' || lower('cefazolin') || '%' then 1
      when lower(label) like '%' || lower('cedax') || '%' then 1
      when lower(label) like '%' || lower('cefoxitin') || '%' then 1
      when lower(label) like '%' || lower('ceftazidime') || '%' then 1
      when lower(label) like '%' || lower('cefaclor') || '%' then 1
      when lower(label) like '%' || lower('cefadroxil') || '%' then 1
      when lower(label) like '%' || lower('cefdinir') || '%' then 1
      when lower(label) like '%' || lower('cefditoren') || '%' then 1
      when lower(label) like '%' || lower('cefepime') || '%' then 1
      when lower(label) like '%' || lower('cefotetan') || '%' then 1
      when lower(label) like '%' || lower('cefotaxime') || '%' then 1
      when lower(label) like '%' || lower('cefpodoxime') || '%' then 1
      when lower(label) like '%' || lower('cefprozil') || '%' then 1
      when lower(label) like '%' || lower('ceftibuten') || '%' then 1
      when lower(label) like '%' || lower('ceftin') || '%' then 1
      when lower(label) like '%' || lower('cefuroxime ') || '%' then 1
      when lower(label) like '%' || lower('cefuroxime') || '%' then 1
      when lower(label) like '%' || lower('cephalexin') || '%' then 1
      when lower(label) like '%' || lower('chloramphenicol') || '%' then 1
      when lower(label) like '%' || lower('cipro') || '%' then 1
      when lower(label) like '%' || lower('ciprofloxacin') || '%' then 1
      when lower(label) like '%' || lower('claforan') || '%' then 1
      when lower(label) like '%' || lower('clarithromycin') || '%' then 1
      when lower(label) like '%' || lower('cleocin') || '%' then 1
      when lower(label) like '%' || lower('clindamycin') || '%' then 1
      when lower(label) like '%' || lower('cubicin') || '%' then 1
      when lower(label) like '%' || lower('dicloxacillin') || '%' then 1
      when lower(label) like '%' || lower('doryx') || '%' then 1
      when lower(label) like '%' || lower('doxycycline') || '%' then 1
      when lower(label) like '%' || lower('duricef') || '%' then 1
      when lower(label) like '%' || lower('dynacin') || '%' then 1
      when lower(label) like '%' || lower('ery-tab') || '%' then 1
      when lower(label) like '%' || lower('eryped') || '%' then 1
      when lower(label) like '%' || lower('eryc') || '%' then 1
      when lower(label) like '%' || lower('erythrocin') || '%' then 1
      when lower(label) like '%' || lower('erythromycin') || '%' then 1
      when lower(label) like '%' || lower('factive') || '%' then 1
      when lower(label) like '%' || lower('flagyl') || '%' then 1
      when lower(label) like '%' || lower('fortaz') || '%' then 1
      when lower(label) like '%' || lower('furadantin') || '%' then 1
      when lower(label) like '%' || lower('garamycin') || '%' then 1
      when lower(label) like '%' || lower('gentamicin') || '%' then 1
      when lower(label) like '%' || lower('kanamycin') || '%' then 1
      when lower(label) like '%' || lower('keflex') || '%' then 1
      when lower(label) like '%' || lower('ketek') || '%' then 1
      when lower(label) like '%' || lower('levaquin') || '%' then 1
      when lower(label) like '%' || lower('levofloxacin') || '%' then 1
      when lower(label) like '%' || lower('lincocin') || '%' then 1
      when lower(label) like '%' || lower('macrobid') || '%' then 1
      when lower(label) like '%' || lower('macrodantin') || '%' then 1
      when lower(label) like '%' || lower('maxipime') || '%' then 1
      when lower(label) like '%' || lower('mefoxin') || '%' then 1
      when lower(label) like '%' || lower('metronidazole') || '%' then 1
      when lower(label) like '%' || lower('minocin') || '%' then 1
      when lower(label) like '%' || lower('minocycline') || '%' then 1
      when lower(label) like '%' || lower('monodox') || '%' then 1
      when lower(label) like '%' || lower('monurol') || '%' then 1
      when lower(label) like '%' || lower('morgidox') || '%' then 1
      when lower(label) like '%' || lower('moxatag') || '%' then 1
      when lower(label) like '%' || lower('moxifloxacin') || '%' then 1
      when lower(label) like '%' || lower('myrac') || '%' then 1
      when lower(label) like '%' || lower('nafcillin sodium') || '%' then 1
      when lower(label) like '%' || lower('nicazel doxy 30') || '%' then 1
      when lower(label) like '%' || lower('nitrofurantoin') || '%' then 1
      when lower(label) like '%' || lower('noroxin') || '%' then 1
      when lower(label) like '%' || lower('ocudox') || '%' then 1
      when lower(label) like '%' || lower('ofloxacin') || '%' then 1
      when lower(label) like '%' || lower('omnicef') || '%' then 1
      when lower(label) like '%' || lower('oracea') || '%' then 1
      when lower(label) like '%' || lower('oraxyl') || '%' then 1
      when lower(label) like '%' || lower('oxacillin') || '%' then 1
      when lower(label) like '%' || lower('pc pen vk') || '%' then 1
      when lower(label) like '%' || lower('pce dispertab') || '%' then 1
      when lower(label) like '%' || lower('panixine') || '%' then 1
      when lower(label) like '%' || lower('pediazole') || '%' then 1
      when lower(label) like '%' || lower('penicillin') || '%' then 1
      when lower(label) like '%' || lower('periostat') || '%' then 1
      when lower(label) like '%' || lower('pfizerpen') || '%' then 1
      when lower(label) like '%' || lower('piperacillin') || '%' then 1
      when lower(label) like '%' || lower('tazobactam') || '%' then 1
      when lower(label) like '%' || lower('primsol') || '%' then 1
      when lower(label) like '%' || lower('proquin') || '%' then 1
      when lower(label) like '%' || lower('raniclor') || '%' then 1
      when lower(label) like '%' || lower('rifadin') || '%' then 1
      when lower(label) like '%' || lower('rifampin') || '%' then 1
      when lower(label) like '%' || lower('rocephin') || '%' then 1
      when lower(label) like '%' || lower('smz-tmp') || '%' then 1
      when lower(label) like '%' || lower('septra') || '%' then 1
      when lower(label) like '%' || lower('septra ds') || '%' then 1
      when lower(label) like '%' || lower('septra') || '%' then 1
      when lower(label) like '%' || lower('solodyn') || '%' then 1
      when lower(label) like '%' || lower('spectracef') || '%' then 1
      when lower(label) like '%' || lower('streptomycin sulfate') || '%' then 1
      when lower(label) like '%' || lower('sulfadiazine') || '%' then 1
      when lower(label) like '%' || lower('sulfamethoxazole') || '%' then 1
      when lower(label) like '%' || lower('trimethoprim') || '%' then 1
      when lower(label) like '%' || lower('sulfatrim') || '%' then 1
      when lower(label) like '%' || lower('sulfisoxazole') || '%' then 1
      when lower(label) like '%' || lower('suprax') || '%' then 1
      when lower(label) like '%' || lower('synercid') || '%' then 1
      when lower(label) like '%' || lower('tazicef') || '%' then 1
      when lower(label) like '%' || lower('tetracycline') || '%' then 1
      when lower(label) like '%' || lower('timentin') || '%' then 1
      when lower(label) like '%' || lower('tobi') || '%' then 1
      when lower(label) like '%' || lower('tobramycin') || '%' then 1
      when lower(label) like '%' || lower('trimethoprim') || '%' then 1
      when lower(label) like '%' || lower('unasyn') || '%' then 1
      when lower(label) like '%' || lower('vancocin') || '%' then 1
      when lower(label) like '%' || lower('vancomycin') || '%' then 1
      when lower(label) like '%' || lower('vantin') || '%' then 1
      when lower(label) like '%' || lower('vibativ') || '%' then 1
      when lower(label) like '%' || lower('vibra-tabs') || '%' then 1
      when lower(label) like '%' || lower('vibramycin') || '%' then 1
      when lower(label) like '%' || lower('zinacef') || '%' then 1
      when lower(label) like '%' || lower('zithromax') || '%' then 1
      when lower(label) like '%' || lower('zmax') || '%' then 1
      when lower(label) like '%' || lower('zosyn') || '%' then 1
      when lower(label) like '%' || lower('zyvox') || '%' then 1
    else 0
    end as antibiotic
  from d_items di
  where linksto in ('inputevents_mv','inputevents_cv')
)
"""

The above (crudely) extracts all possible antibiotics in the data. Now we can isolate antibiotic administrations from `inputevents_mv` and `inputevents_cv`.


In [6]:
# metavision query
query_abx_iv_mv = """
, mv as
(
  select icustay_id
  , label as first_antibiotic_name
  , starttime as first_antibiotic_time
  , ROW_NUMBER() over (partition by icustay_id order by starttime, endtime) as rn
  from inputevents_mv mv
  inner join di
      on mv.itemid = di.itemid
      and di.antibiotic = 1
  where statusdescription != 'Rewritten'
)
"""

query_abx_iv_cv = """
, cv as
(
  select icustay_id
  , label as first_antibiotic_name
  , charttime as first_antibiotic_time
  , ROW_NUMBER() over (partition by icustay_id order by charttime) as rn
  from inputevents_cv cv
  inner join di
      on cv.itemid = di.itemid
      and di.antibiotic = 1
)
"""

# join these two together
query_abtbl = \
"""
, ab_tbl as
(
select
      ie.icustay_id, ie.hadm_id
    , case when mv.first_antibiotic_time is not null and cv.first_antibiotic_time is not null
        then case when mv.first_antibiotic_time < cv.first_antibiotic_time 
                then mv.first_antibiotic_name
            else cv.first_antibiotic_name
            end
        else coalesce(mv.first_antibiotic_name, cv.first_antibiotic_name)
     end first_antibiotic_name
    , case when mv.first_antibiotic_time is not null and cv.first_antibiotic_time is not null
        then case when mv.first_antibiotic_time < cv.first_antibiotic_time 
                then mv.first_antibiotic_time
            else cv.first_antibiotic_time
            end
        else coalesce(mv.first_antibiotic_time, cv.first_antibiotic_time)
     end first_antibiotic_time
from icustays ie
left join mv
    on ie.icustay_id = mv.icustay_id
    and mv.rn = 1
left join cv
    on ie.icustay_id = cv.icustay_id
    and cv.rn = 1
)
"""

query_abx_iv = query_ab_ditems + query_abx_iv_mv + query_abx_iv_cv + query_abtbl

# select all from the last temp view
abx_iv = pd.read_sql_query(query_schema + query_abx_iv + 'select * from ab_tbl',con)
abx_iv.describe(include='all')

,icustay_id,hadm_id,first_antibiotic_name,first_antibiotic_time
count,61532.000000,61532.000000,14235,14235
unique,NaN,NaN,26,14212
top,NaN,NaN,Vancomycin,2162-04-12 20:00:00
freq,NaN,NaN,4229,2
mean,249962.710248,149954.470649,NaN,NaN
std,28890.574867,28898.895904,NaN,NaN
min,200001.000000,100001.000000,NaN,NaN
25%,224935.500000,124933.750000,NaN,NaN
50%,249940.000000,149911.500000,NaN,NaN
75%,274972.500000,174997.250000,NaN,NaN


# Defining suspected infection

Suspected infection is defined as:

* Antibiotics within 72 hours of a culture
* A culture within 24 hours of antibiotics

So the next step is to left join to this table from the antibiotic table with the following rules:

* If first_antibiotic_time is null, do not join
* If charttime is null, use chartdate and add an extra day to allow for fuzziness
* If charttime is not null, use charttime

Here's the first stab:

In [7]:
# add micro table
query_abx_iv_me = query_abx_iv + ", me as ( " + query_me + " ) "

query = query_abx_iv_me + \
"""
select 
  ab_tbl.*
  , me72.spec_type_desc as last72_specimen
  , coalesce(me72.charttime, me72.chartdate) as last72_time
  , me72.PositiveCulture as last72_positive
  , me24.spec_type_desc as next24_specimen
  , coalesce(me24.charttime, me24.chartdate)as next24_time
  , me24.PositiveCulture as next24_positive
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
"""

ab = pd.read_sql_query(query_schema + query,con)
ab.head()

,icustay_id,hadm_id,first_antibiotic_name,first_antibiotic_time,last72_specimen,last72_time,last72_positive,next24_specimen,next24_time,next24_positive
0,275225,100001,None,NaT,None,NaT,NaN,None,NaT,NaN
1,209281,100003,Vancomycin,2150-04-17 22:14:00,BLOOD CULTURE,2150-04-17 18:41:00,0.0,URINE,2150-04-17 23:01:00,0.0
2,209281,100003,Vancomycin,2150-04-17 22:14:00,MRSA SCREEN,2150-04-17 19:00:00,0.0,URINE,2150-04-17 23:01:00,0.0
3,209281,100003,Vancomycin,2150-04-17 22:14:00,BLOOD CULTURE,2150-04-17 18:41:00,0.0,BLOOD CULTURE,2150-04-17 23:01:00,0.0
4,209281,100003,Vancomycin,2150-04-17 22:14:00,MRSA SCREEN,2150-04-17 19:00:00,0.0,BLOOD CULTURE,2150-04-17 23:01:00,0.0


For now, let's just define it based of the antibiotic dose, and look for *all* cultures before/after. This will not affect the definition of suspected infection (as the definition only requires one).

We'll group and count how often charttime is available, and how often chartdate is available. We're a bit worried about quantization due to chartdate.

In [8]:
query = query_abx_iv_me + \
"""
, ab_fnl as
(
select 
  ab_tbl.icustay_id
  , ab_tbl.first_antibiotic_name
  , ab_tbl.first_antibiotic_time
  , min(me72.charttime) as last72_charttime
  , min(me72.chartdate) as last72_chartdate
  , min(me24.charttime) as next24_charttime
  , min(me24.chartdate) as next24_chartdate
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
group by ab_tbl.icustay_id, ab_tbl.first_antibiotic_name, ab_tbl.first_antibiotic_time
)
select
  count(icustay_id) as NumICU
  , count(first_antibiotic_name) as HasAB
  , count(last72_charttime) as last72_charttime
  , count(last72_chartdate) as last72_chartdate
  , count(case when last72_charttime is null then last72_chartdate end) as last72_chartdate_only
  , count(next24_charttime) as next24_charttime
  , count(next24_chartdate) as next24_chartdate
  , count(case when next24_charttime is null then next24_chartdate end) as next24_chartdate_only
from ab_fnl
"""

ab = pd.read_sql_query(query_schema + query,con)
ab.head()

,numicu,hasab,last72_charttime,last72_chartdate,last72_chartdate_only,next24_charttime,next24_chartdate,next24_chartdate_only
0,61532,14235,12994,13119,125,6934,6934,0


This tells us something: if the culture was drawn in the ICU (as it would be for next 24 hours), then it always has a charttime. But if it's drawn before the ICU, then it is (very infrequently) missing a charttime. Lets look at these chartdates.

In [9]:
query = query_abx_iv_me + \
"""
, ab_fnl as
(
select 
  ab_tbl.icustay_id
  , ab_tbl.first_antibiotic_name
  , ab_tbl.first_antibiotic_time
  , min(me72.charttime) as last72_charttime
  , min(me72.chartdate) as last72_chartdate
  , min(me24.charttime) as next24_charttime
  , min(me24.chartdate) as next24_chartdate
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
group by ab_tbl.icustay_id, ab_tbl.first_antibiotic_name, ab_tbl.first_antibiotic_time
)
select
  ab_fnl.*
from ab_fnl
where last72_charttime is null and last72_chartdate is not null;
"""

ab = pd.read_sql_query(query_schema + query,con)
ab.head(n=10)

,icustay_id,first_antibiotic_name,first_antibiotic_time,last72_charttime,last72_chartdate,next24_charttime,next24_chartdate
0,206836,Vancomycin,2120-05-01 07:20:00,None,2120-05-01,2120-05-01 10:01:00,2120-05-01
1,266120,Erythromycin,2132-04-10 16:30:00,None,2132-04-10,2132-04-10 18:56:00,2132-04-10
2,227420,Cefazolin,2111-03-16 11:00:00,None,2111-03-16,2111-03-16 19:36:00,2111-03-16
3,226823,Levofloxacin,2170-05-11 00:00:00,None,2170-05-10,2170-05-11 02:43:00,2170-05-11
4,286447,Metronidazole,2144-04-08 00:00:00,None,2144-04-07,NaT,NaT
5,218042,Cefazolin,2113-08-10 20:54:00,None,2113-08-10,NaT,NaT
6,261807,Cefazolin,2144-03-21 00:37:00,None,2144-03-20,2144-03-21 01:54:00,2144-03-21
7,270629,Piperacillin,2119-10-24 00:00:00,None,2119-10-23,NaT,NaT
8,224106,Vancomycin,2179-03-10 20:40:00,None,2179-03-10,2179-03-10 21:26:00,2179-03-10
9,226506,Metronidazole,2170-04-05 00:10:00,None,2170-04-05,2170-04-05 01:37:00,2170-04-05


For these cases, let's assume the first antibiotic time is the starting time of suspicion of infection.

With this rule, we can create a table for the antibiotic definition using IV.

In [10]:
query = query_schema + \
"""
DROP TABLE IF EXISTS abx_micro_iv CASCADE;
CREATE TABLE abx_micro_iv as
""" + query_abx_iv_me + \
"""
, ab_fnl as
(
select 
  ab_tbl.icustay_id
  , ab_tbl.first_antibiotic_name
  , ab_tbl.first_antibiotic_time
  , me72.charttime as last72_charttime
  , me72.chartdate as last72_chartdate
  , me24.charttime as next24_charttime
  , me24.chartdate as next24_chartdate
  
  , me72.positiveculture as last72_positiveculture
  , me72.spec_type_desc as last72_specimen
  , me24.positiveculture as next24_positiveculture
  , me24.spec_type_desc as next24_specimen
  
  , ROW_NUMBER() over (partition by ab_tbl.icustay_id order by coalesce(me72.charttime, me24.charttime, me72.chartdate))
      as rn
from ab_tbl
-- blood culture in last 72 hours
left join me me72
  on ab_tbl.hadm_id = me72.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and 
  ( 
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me72.charttime 
    and ab_tbl.first_antibiotic_time <= me72.charttime + interval '72' hour
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me72.charttime is null
    and ab_tbl.first_antibiotic_time > me72.chartdate 
    and ab_tbl.first_antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
    )
  )
-- blood culture in subsequent 24 hours
left join me me24
  on ab_tbl.hadm_id = me24.hadm_id
  and ab_tbl.first_antibiotic_time is not null
  and me24.charttime is not null
  and
  (
    -- if charttime is available, use it
    (
        ab_tbl.first_antibiotic_time > me24.charttime - interval '24' hour
    and ab_tbl.first_antibiotic_time <= me24.charttime 
    )
    OR
    (
    -- if charttime is not available, use chartdate
        me24.charttime is null
    and ab_tbl.first_antibiotic_time > me24.chartdate
    and ab_tbl.first_antibiotic_time <= me24.chartdate + interval '24' hour
    )
  )
)
select
  ab_fnl.icustay_id
  
  , first_antibiotic_name as antibiotic_name
  , first_antibiotic_time as antibiotic_time
  , coalesce(last72_charttime, last72_chartdate) as last72_charttime
  , coalesce(next24_charttime, next24_chartdate) as next24_charttime
  
  -- time of suspected infection: either the culture time (if before antibiotic), or the antibiotic time
  , case 
      when last72_charttime is not null
        then last72_charttime
      when next24_charttime is not null or last72_chartdate is not null
        then first_antibiotic_time
    else null
  end as suspected_infection_time
  -- the specimen that was cultured
  , case 
      when last72_charttime is not null or last72_chartdate is not null
        then last72_specimen
      when next24_charttime is not null 
        then next24_specimen
    else null
  end as specimen
  -- whether the cultured specimen ended up being positive or not
  , case 
      when last72_charttime is not null or last72_chartdate is not null
        then last72_positiveculture
      when next24_charttime is not null 
        then next24_positiveculture
    else null
  end as positiveculture
from ab_fnl
where rn = 1
order by icustay_id;
"""

cur = con.cursor()
cur.execute(query)
cur.execute('COMMIT;')
cur.close()

print('abx_micro_iv table created!')

abx_micro_iv table created!


## Antibiotics using `prescriptions`

Now we follow the same logic, except we will define antibiotics using `prescriptions`.

First, we call a query available in this repository that generates a list of antibiotics available in the `prescriptions` table. This query can take some time, so we add some code that checks if the table already exists, and only creates the table if we need to.

In [11]:
# check if abx_poe_list table exists
# "abx_poe_list"
#  abx == antibiotics
#  poe == provider order entry == the hospital interface that doctors use to prescribe medications

query = query_schema + \
"select * from information_schema.tables where table_name = 'abx_poe_list'"
df = pd.read_sql_query(query,con)

if df.shape[0]>0:
    print('Table exists - skipping regeneration of table.')
else:
    # read the query from the file into a single string
    with open('../query/tbls/abx-poe-list.sql') as fp:
        query = fp.readlines()
    
    # call the query to create the table using a cursor
    cur = con.cursor()
    cur.execute(query_schema)
    cur.execute(query)
    cur.execute('COMMIT;')
    cur.close()
    print('abx_poe_list table created!')

Table exists - skipping regeneration of table.


In [12]:
# load in the table
query = query_schema + \
"select * from abx_poe_list;"
abx_poe = pd.read_sql_query(query, con)
abx_poe.head()

,drug,numobs
0,Vancomycin,42633
1,Vancomycin HCl,22291
2,Levofloxacin,20867
3,MetRONIDAZOLE (FLagyl),9985
4,Piperacillin-Tazobactam Na,9275


Here we can see a list of the antibiotics along with a count of how frequently that antibiotic occurred in the prescription table. Vancomycin is by far the most common, though it is not consistently documented using the same drug name. Just for reference: the query to find these antibiotics was generated using regular expressions and then manually reviewed by a clinical data scientist and an intensivist.

With this table in hand, we can easily isolate prescribed antibiotics for all patients using an inner join:

In [13]:
query_abx_poe = """
  select pr.hadm_id
  , pr.drug as antibiotic_name
  , pr.startdate as antibiotic_time
  , pr.enddate as antibiotic_endtime
  from prescriptions pr
  -- inner join to subselect to only antibiotic prescriptions
  inner join abx_poe_list ab
      on pr.drug = ab.drug
"""

Now we recreate the rules as outlined by sepsis-3:

* antibiotics up to 72 hours *before* blood culture
* blood culture up to 24 hours *before* antibiotics

This query would be relatively straightforward, except we only have prescriptions on a daily base and sometimes blood cultures are only recorded with the day resolution. In these cases, we have decided to be a bit more flexible with the time windows by extending them 24 hours. The result is as follows: if I have a blood culture on `01-20 00:00`, then the query looks back *96* hours, not *72*, which allows a prescription dated `01-17 00:00` to be included. Note that the prescription dated `01-17 00:00` was likely done sometime during the day, and so this rule is conservative.

In [14]:
query = query_schema + \
"""
DROP TABLE IF EXISTS abx_micro_poe CASCADE;
CREATE TABLE abx_micro_poe as
with abx as
(
""" + query_abx_poe + """
)
-- get cultures for each icustay
-- note this duplicates prescriptions
-- each ICU stay in the same hospitalization will get a copy of all prescriptions for that hospitalization
, ab_tbl as
(
  select
        ie.subject_id, ie.hadm_id, ie.icustay_id
      , ie.intime, ie.outtime
      , abx.antibiotic_name
      , abx.antibiotic_time
      , abx.antibiotic_endtime
  from icustays ie
  left join abx
      on ie.hadm_id = abx.hadm_id
)
, me as
(
""" + query_me + """
)
, ab_fnl as
(
  select
      ab_tbl.icustay_id, ab_tbl.intime, ab_tbl.outtime
    , ab_tbl.antibiotic_name
    , ab_tbl.antibiotic_time
    , coalesce(me72.charttime,me72.chartdate) as last72_charttime
    , coalesce(me24.charttime,me24.chartdate) as next24_charttime
    , me72.positiveculture as last72_positiveculture
    , me72.spec_type_desc as last72_specimen
    , me24.positiveculture as next24_positiveculture
    , me24.spec_type_desc as next24_specimen
  from ab_tbl
  -- blood culture in last 72 hours
  left join me me72
    on ab_tbl.hadm_id = me72.hadm_id
    and ab_tbl.antibiotic_time is not null
    and
    (
      -- if charttime is available, use it
      (
          ab_tbl.antibiotic_time > me72.charttime
      and ab_tbl.antibiotic_time <= me72.charttime + interval '72' hour
      )
      OR
      (
      -- if charttime is not available, use chartdate
          me72.charttime is null
      and ab_tbl.antibiotic_time > me72.chartdate
      and ab_tbl.antibiotic_time < me72.chartdate + interval '96' hour -- could equally do this with a date_trunc, but that's less portable
      )
    )
  -- blood culture in subsequent 24 hours
  left join me me24
    on ab_tbl.hadm_id = me24.hadm_id
    and ab_tbl.antibiotic_time is not null
    and me24.charttime is not null
    and
    (
      -- if charttime is available, use it
      (
          ab_tbl.antibiotic_time > me24.charttime - interval '24' hour
      and ab_tbl.antibiotic_time <= me24.charttime
      )
      OR
      (
      -- if charttime is not available, use chartdate
          me24.charttime is null
      and ab_tbl.antibiotic_time > me24.chartdate
      and ab_tbl.antibiotic_time <= me24.chartdate + interval '24' hour
      )
    )
)
, ab_laststg as
(
select
  icustay_id
  , antibiotic_name
  , antibiotic_time
  , last72_charttime
  , next24_charttime

  -- time of suspected infection: either the culture time (if before antibiotic), or the antibiotic time
  , case
      when coalesce(last72_charttime,next24_charttime) is null
        then 0
      else 1 end as suspected_infection

  , coalesce(last72_charttime,next24_charttime) as suspected_infection_time

  -- the specimen that was cultured
  , case
      when last72_charttime is not null
        then last72_specimen
      when next24_charttime is not null
        then next24_specimen
    else null
  end as specimen

  -- whether the cultured specimen ended up being positive or not
  , case
      when last72_charttime is not null
        then last72_positiveculture
      when next24_charttime is not null
        then next24_positiveculture
    else null
  end as positiveculture
  
  -- used to identify the *first* occurrence of suspected infection
  , ROW_NUMBER() over 
  (
     PARTITION BY ab_fnl.icustay_id
     ORDER BY coalesce(last72_charttime, next24_charttime)
  )
      as rn
from ab_fnl
)
select
  icustay_id
  , antibiotic_name
  , antibiotic_time
  , last72_charttime
  , next24_charttime
  , suspected_infection_time
  , specimen, positiveculture
from ab_laststg
where rn=1
order by icustay_id, antibiotic_time;
"""

cur = con.cursor()
cur.execute(query)
cur.execute('COMMIT;')
cur.close()

print('abx_micro_poe table created!')

abx_micro_poe table created!


# Compare definitions

First, let's load the data in and look at a few rows.

In [15]:
df_abx_poe = pd.read_sql_query(query_schema + "select * from abx_micro_poe", con)
df_abx_iv = pd.read_sql_query(query_schema + "select * from abx_micro_iv", con)

In [16]:
print('Prescriptions')
display(HTML(df_abx_poe.head().to_html()))
print('IV')
display(HTML(df_abx_iv.head().to_html()))

Prescriptions


,icustay_id,antibiotic_name,antibiotic_time,last72_charttime,next24_charttime,suspected_infection_time,specimen,positiveculture
0,200001,Vancomycin,2181-11-19,2181-11-18 11:10:00,2181-11-19 10:17:00,2181-11-18 11:10:00,BLOOD CULTURE,0.0
1,200003,Piperacillin-Tazobactam Na,2199-08-02,NaT,2199-08-02 21:02:00,2199-08-02 21:02:00,BLOOD CULTURE,1.0
2,200006,None,NaT,NaT,NaT,NaT,None,NaN
3,200007,None,NaT,NaT,NaT,NaT,None,NaN
4,200009,Vancomycin,2189-11-30,NaT,NaT,NaT,None,NaN


IV


,icustay_id,antibiotic_name,antibiotic_time,last72_charttime,next24_charttime,suspected_infection_time,specimen,positiveculture
0,200001,None,None,None,None,None,None,NaN
1,200003,None,None,None,None,None,None,NaN
2,200006,None,None,None,None,None,None,NaN
3,200007,None,None,None,None,None,None,NaN
4,200009,None,None,None,None,None,None,NaN


Right away we can see there may be a substantial disagreement in the frequency.

In [17]:
print('Prescriptions: {}'.format(df_abx_poe['suspected_infection_time'].count()))
print('Intravenous:   {}'.format(df_abx_iv['suspected_infection_time'].count()))

Prescriptions: 35389
Intravenous:   13689


On a hunch, let's stratify this based off year: 2001-2008 ('carevue') and 2008-2012 ('metavision').

In [18]:
query_add_dbsource = " abx inner join icustays ie on abx.icustay_id = ie.icustay_id"
df_abx_poe = pd.read_sql_query(query_schema + "select abx.*, ie.dbsource from abx_micro_poe" + query_add_dbsource, con)
df_abx_iv = pd.read_sql_query(query_schema + "select abx.*, ie.dbsource from abx_micro_iv" + query_add_dbsource, con)

In [19]:
print('Prescriptions')
print(df_abx_poe.groupby('dbsource')['suspected_infection_time'].count())

print('')
print('Intravenous')
print(df_abx_iv.groupby('dbsource')['suspected_infection_time'].count())

Prescriptions
dbsource
both            117
carevue       18423
metavision    16849
Name: suspected_infection_time, dtype: int64

Intravenous
dbsource
both             62
carevue          25
metavision    13602
Name: suspected_infection_time, dtype: int64


Aha!

It seems that we do not capture any antibiotic usage from the earlier database system carevue when we use the intravenous table `inputevents_cv` (see https://mimic.physionet.org/mimicdata/carevue/ for a discussion of the CareVue database system).

It's also worth noting that we in general have much fewer entries identified in metavision too - we somewhat expect this - again remember prescriptions is hospital wide but intravenous administrations is ICU only.

# Conclusion

We will use the `abx_micro_poe` table as this has the advantage of (i) capturing antibiotic administration outside the ICU and (ii) capturing antibiotic administration for earlier admissions.

Queries are available to do this, and do not need to be run in this notebook. Simply run (in order):

* `abx-poe-list.sql` - found in `query/tbls` subfolder of this repository
* `abx-micro-prescription.sql` - found in `query/tbls` subfolder in this repository

These scripts are identical to the code presented above.